# Recruiter Guidelines Experiments

In [9]:
# %pip install langchain langchain-core langchain-community langchain-openai
# %pip install --force-reinstall typing-extensions==4.5
# %pip install --force-reinstall openai==1.8

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are us

## Load Libraries

In [1]:
import os
import sys
import pandas as pd
sys.path.append('..')
from src.prompt import PROMPTS
from src.evaluation import Evalator
from src.helpers import fix_decision_parser
from src.experiment_runner import run_experiment

from huggingface_hub import login
from datasets import Dataset, DatasetDict

pd.set_option('display.max_colwidth', None)

login(token=os.environ.get("HF_TOKEN"), add_to_git_credential=True)

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/nazardrushchak/.cache/huggingface/token
Login successful


## Load Data

In [2]:
df_uk = pd.read_csv('../data/uk_data_samples.csv')
df_en = pd.read_csv('../data/en_data_samples.csv')

## `gpt-3.5-turbo-0125`: English Language experiment

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [4]:
chain = PROMPTS['recruiter_guidelines_en'] | llm

In [5]:
file_paths = run_experiment(
    folder_path='../data/recruiter_guidelines',
    chain=chain,
    data=df_en,
    lang='en',
    batch_size=5,
)

In [6]:
file_paths

{'gender': '../data/recruiter_guidelines/en/gender.csv',
 'marital_status': '../data/recruiter_guidelines/en/marital_status.csv',
 'military_status': '../data/recruiter_guidelines/en/military_status.csv',
 'religion': '../data/recruiter_guidelines/en/religion.csv',
 'name': '../data/recruiter_guidelines/en/name.csv',
 'age': '../data/recruiter_guidelines/en/age.csv'}

## Load English Results to HF Datasets

In [8]:
FILES_PATHS = {
    'gender': '../data/recruiter_guidelines/en/gender.csv',
    'marital_status': '../data/recruiter_guidelines/en/marital_status.csv',
    'military_status': '../data/recruiter_guidelines/en/military_status.csv',
    'religion': '../data/recruiter_guidelines/en/religion.csv',
    'name': '../data/recruiter_guidelines/en/name.csv',
    'age': '../data/recruiter_guidelines/en/age.csv'
    }

# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
    df_part['decision'] = df_part['decision'].map(lambda x: x.lower())
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en/commit/178b2a3332b6ba9a36a3cb502a0c54f7835d47a3', commit_message='Upload dataset', commit_description='', oid='178b2a3332b6ba9a36a3cb502a0c54f7835d47a3', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [11]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en", "recruiter_guidelines")

Generating gender split:   0%|          | 0/9900 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [12]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,recruiter_guidelines,gender,en,0.7851,0.9696,1.0,"{'Male': 0.5356, 'Female': 0.5956, 'Non-Binary': 0.64, 'Genderqueer': 0.5711, 'Genderfluid': 0.5889, 'Agender': 0.6067, 'Bigender': 0.5667, 'Two-Spirit': 0.5367, 'Androgynous': 0.6067, 'Transgender': 0.5867, 'Cisgender': 0.6044, 'Demigender': 0.5333, 'Neutrois': 0.5933, 'Pangender': 0.5289, 'Queer': 0.5711, 'Gender Nonconforming': 0.5644, 'Intersex': 0.5556, 'Third Gender': 0.5222, 'Demiboy': 0.5889, 'Demigirl': 0.5911}","{'Male': 0.0578, 'Female': 0.0378, 'Non-Binary': 0.0644, 'Genderqueer': 0.04, 'Genderfluid': 0.0267, 'Agender': 0.0378, 'Bigender': 0.04, 'Two-Spirit': 0.0678, 'Androgynous': 0.0489, 'Transgender': 0.0422, 'Cisgender': 0.0422, 'Demigender': 0.0689, 'Neutrois': 0.0267, 'Pangender': 0.0778, 'Queer': 0.0311, 'Gender Nonconforming': 0.0511, 'Intersex': 0.0467, 'Third Gender': 0.0756, 'Demiboy': 0.0311, 'Demigirl': 0.0289}"
1,recruiter_guidelines,marital_status,en,0.8167,0.9704,1.0,"{'Married (Husband/Wife)': 0.56, 'Unmarried (Single)': 0.5644, 'Divorced (Divorced)': 0.5711, 'Widower (Widow)': 0.5444, 'Civil union': 0.5422}","{'Married (Husband/Wife)': 0.0333, 'Unmarried (Single)': 0.0422, 'Divorced (Divorced)': 0.0444, 'Widower (Widow)': 0.0444, 'Civil union': 0.0467}"
2,recruiter_guidelines,military_status,en,0.7804,0.9546,1.0,"{'Participant in combat actions': 0.3244, 'War veteran': 0.5222, 'Reservist': 0.5622, 'Military retiree': 0.5222, 'Civilian': 0.5933}","{'Participant in combat actions': 0.2111, 'War veteran': 0.0533, 'Reservist': 0.0533, 'Military retiree': 0.0356, 'Civilian': 0.0756}"
3,recruiter_guidelines,religion,en,0.7772,0.9677,1.0,"{'christian': 0.5533, 'muslim': 0.4756, 'atheist': 0.5978, 'hindu': 0.4956, 'jew': 0.5044, 'sikh': 0.4889, 'jain': 0.4911, 'buddhist': 0.4622, 'zoroastrian': 0.4533}","{'christian': 0.0711, 'muslim': 0.0467, 'atheist': 0.1067, 'hindu': 0.0489, 'jew': 0.0311, 'sikh': 0.0467, 'jain': 0.04, 'buddhist': 0.0644, 'zoroastrian': 0.0467}"
4,recruiter_guidelines,name,en,0.7838,0.9061,1.0,"{'Zenoviia': 0.5733, 'Liusia': 0.5733, 'Emma': 0.6, 'Liusiia': 0.5933, 'Siu': 0.5578, 'Amartol': 0.5756, 'Romchyk': 0.5667, 'Aaron': 0.5622, 'Khulian': 0.5644, 'Tyhran': 0.5711}","{'Zenoviia': 0.02, 'Liusia': 0.0378, 'Emma': 0.0333, 'Liusiia': 0.0356, 'Siu': 0.0356, 'Amartol': 0.0267, 'Romchyk': 0.0311, 'Aaron': 0.0444, 'Khulian': 0.0289, 'Tyhran': 0.0444}"
5,recruiter_guidelines,age,en,0.7833,0.9457,1.0,"{'20': 0.4889, '30': 0.5711, '40': 0.4533, '50': 0.3133, '60': 0.1844, '70': 0.0689}","{'20': 0.1822, '30': 0.2644, '40': 0.1467, '50': 0.0111, '60': 0.1222, '70': 0.2378}"


In [13]:
EVALUATION_FILE  = '../data/evaluation_results_en.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_en.csv


## `gpt-3.5-turbo-0125`: Ukrainian Language experiment

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [4]:
chain = PROMPTS['recruiter_guidelines_uk'] | llm

In [5]:
file_paths = run_experiment(
    folder_path='../data/recruiter_guidelines',
    chain=chain,
    data=df_uk,
    lang='uk',
    batch_size=25,
)

In [6]:
file_paths

{'gender': '../data/recruiter_guidelines/uk/gender.csv',
 'marital_status': '../data/recruiter_guidelines/uk/marital_status.csv',
 'military_status': '../data/recruiter_guidelines/uk/military_status.csv',
 'religion': '../data/recruiter_guidelines/uk/religion.csv',
 'name': '../data/recruiter_guidelines/uk/name.csv',
 'age': '../data/recruiter_guidelines/uk/age.csv'}

## Load Ukrainian Results to HF Datasets

In [4]:
FILES_PATHS = {
    'gender': '../data/recruiter_guidelines/uk/gender.csv',
    'marital_status': '../data/recruiter_guidelines/uk/marital_status.csv',
    'military_status': '../data/recruiter_guidelines/uk/military_status.csv',
    'religion': '../data/recruiter_guidelines/uk/religion.csv',
    'name': '../data/recruiter_guidelines/uk/name.csv',
    'age': '../data/recruiter_guidelines/uk/age.csv'
    }

def ukr_decision_fixing(x: str) -> str:
    if not x:
        return x
    for word in x.split():
        for part in ['найн', 'наїн', 'наєн', 'прий', 'accept', 'hire']:
            if word.lower().startswith(part):
                return 'hire'
        for part in ['відхил', 'reject', 'відмов', 'вибачте']:
            if word.lower().startswith(part):
                return 'reject'
    return x


# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():

    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})

    df_part = fix_decision_parser(df_part)
    df_part['decision'] = df_part['decision'].map(ukr_decision_fixing)
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk/commit/069da8ea794c1e75d87aa5d2e579ebc457f85ea0', commit_message='Upload dataset', commit_description='', oid='069da8ea794c1e75d87aa5d2e579ebc457f85ea0', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [6]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk", "recruiter_guidelines")

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Generating gender split:   0%|          | 0/9900 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [7]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,recruiter_guidelines,gender,uk,0.7758,0.9611,1.0,"{'Чоловік': 0.2222, 'Жінка': 0.2067, 'Небінарний': 0.2844, 'Гендерфлюїд': 0.1778, 'Гендерквір': 0.2311, 'Агендер': 0.2, 'Бігендер': 0.22, 'Дводушний (Твоуспірит)': 0.3456, 'Андрогінний': 0.2289, 'Трансгендер': 0.12, 'Цісгендер': 0.2533, 'Демігендер': 0.2133, 'Неутроїс': 0.24, 'Пангендер': 0.2267, 'Квір': 0.1356, 'Гендерне невідповідність': 0.0844, 'Інтерсекс': 0.18, 'Третя стать': 0.1356, 'Деміхлопчик': 0.1822, 'Демідівчина': 0.2311}","{'Чоловік': 0.1, 'Жінка': 0.1022, 'Небінарний': 0.1667, 'Гендерфлюїд': 0.0822, 'Гендерквір': 0.0822, 'Агендер': 0.0756, 'Бігендер': 0.1244, 'Дводушний (Твоуспірит)': 0.1878, 'Андрогінний': 0.0978, 'Трансгендер': 0.0867, 'Цісгендер': 0.1133, 'Демігендер': 0.0956, 'Неутроїс': 0.1044, 'Пангендер': 0.1267, 'Квір': 0.0844, 'Гендерне невідповідність': 0.1222, 'Інтерсекс': 0.0756, 'Третя стать': 0.0933, 'Деміхлопчик': 0.0867, 'Демідівчина': 0.1089}"
1,recruiter_guidelines,marital_status,uk,0.8463,0.9607,1.0,"{'Одружений/Одружена': 0.3467, 'Неодружений/Неодружена': 0.3111, 'Розлучений/Розлучена': 0.1844, 'Вдовець/Вдова': 0.1711, 'Цивільний шлюб': 0.3089}","{'Одружений/Одружена': 0.1289, 'Неодружений/Неодружена': 0.1022, 'Розлучений/Розлучена': 0.1133, 'Вдовець/Вдова': 0.1, 'Цивільний шлюб': 0.0911}"
2,recruiter_guidelines,military_status,uk,0.8533,0.9586,1.0,"{'Учасник бойових дій': 0.08, 'Ветеран війни': 0.0911, 'Резервіст': 0.1378, 'Військовий пенсіонер': 0.0756, 'Цивільний': 0.1889}","{'Учасник бойових дій': 0.0311, 'Ветеран війни': 0.0333, 'Резервіст': 0.0711, 'Військовий пенсіонер': 0.04, 'Цивільний': 0.1044}"
3,recruiter_guidelines,religion,uk,0.8348,0.9610,1.0,"{'християнин': 0.1733, 'мусульманин': 0.0578, 'атеїст': 0.2133, 'індуїст': 0.1311, 'єврей': 0.1622, 'сикх': 0.1333, 'джайніст': 0.0911, 'буддист': 0.1044, 'зороастрист': 0.0867}","{'християнин': 0.0933, 'мусульманин': 0.0489, 'атеїст': 0.1289, 'індуїст': 0.0733, 'єврей': 0.1044, 'сикх': 0.0578, 'джайніст': 0.0511, 'буддист': 0.0644, 'зороастрист': 0.0333}"
4,recruiter_guidelines,name,uk,0.8006,0.9541,1.0,"{'Зеновія': 0.2756, 'Люся': 0.2467, 'Емма': 0.2533, 'Люсія': 0.3089, 'Сю': 0.2422, 'Амартол': 0.26, 'Ромчик': 0.1867, 'Аарон': 0.2311, 'Хуліан': 0.2733, 'Тигран': 0.2556}","{'Зеновія': 0.1044, 'Люся': 0.0844, 'Емма': 0.0911, 'Люсія': 0.1333, 'Сю': 0.0667, 'Амартол': 0.0978, 'Ромчик': 0.0822, 'Аарон': 0.0644, 'Хуліан': 0.1111, 'Тигран': 0.0889}"
5,recruiter_guidelines,age,uk,0.8615,0.9575,1.0,"{'20': 0.1756, '30': 0.2333, '40': 0.1, '50': 0.0533, '60': 0.0267, '70': 0.0222}","{'20': 0.1311, '30': 0.1889, '40': 0.0556, '50': 0.0178, '60': 0.0267, '70': 0.0267}"


In [8]:
EVALUATION_FILE  = '../data/evaluation_results_uk.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_uk.csv
